# Carbon - Briefing

# Parte 1 - Os dados

Este trabalho utilizará os dados disponíveis em https://ourworldindata.org/co2-emissions, sobre as emissões de carbono por país e região ao longo dos últimos $300$ anos.

A ideia do projeto é entender quais são os países e regiões que mais emitem carbono, além de como isso variou ao longo do tempo e como as emissões de carbono se relacionam com o desenvolvimento econômico que ocorreu nos países no último século.

As informações que serão utilizadas estão disponíveis, mas não limitadas a, na página de emissão de carbono no site do Our World in Data.

O primeiro dataset utilizado é o "annual-co2-emissions-per-country" que detalha para cada país, continente e para o mundo qual foi a emissão de carbono de cada ano, com dados disponíveis a partir de $1751$.

In [1]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
carbon_country = pd.read_csv('annual-co2-emissions-per-country.csv')

In [3]:
carbon_country.head()

,Entity,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1949,0.014656
1,Afghanistan,AFG,1950,0.084272
2,Afghanistan,AFG,1951,0.091600
3,Afghanistan,AFG,1952,0.091600
4,Afghanistan,AFG,1953,0.106256


# Parte 2 - Visualizações iniciais

## 2.1) Emissões mundiais

In [4]:
global_carbon_year = carbon_country[carbon_country['Entity']=='World']
global_carbon_year

,Entity,Code,Year,Annual CO2 emissions
22751,World,OWID_WRL,1751,9.350528
22752,World,OWID_WRL,1752,9.354192
22753,World,OWID_WRL,1753,9.354192
22754,World,OWID_WRL,1754,9.357856
22755,World,OWID_WRL,1755,9.361520
...,...,...,...,...
23014,World,OWID_WRL,2014,35505.827040
23015,World,OWID_WRL,2015,35462.746740
23016,World,OWID_WRL,2016,35675.099440
23017,World,OWID_WRL,2017,36153.261650


In [5]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Year'], empty='none')

# The basic line
line = alt.Chart(global_carbon_year).mark_line().encode(
    x='Year',
    y='Annual CO2 emissions'
).properties(
    title='Annual CO2 emissions globally'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(global_carbon_year).mark_point().encode(
    x='Year',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='right', dx=-5, dy=-10).encode(
    text=alt.condition(nearest, 'Annual CO2 emissions', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(global_carbon_year).mark_rule(color='gray').encode(
    x='Year',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

## 2.2) Emissões por continente/região

In [10]:
carbon_region = carbon_country.loc[carbon_country['Entity'].isin(['Asia','North America','Europe','Africa','South America','Oceania'])]
carbon_region['Region'] = carbon_region['Entity']
carbon_region = carbon_region.drop(['Entity','Code'], axis=1)
carbon_region.head()

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Annual CO2 emissions,Region
70,1751,0.0,Africa
71,1752,0.0,Africa
72,1753,0.0,Africa
73,1754,0.0,Africa
74,1755,0.0,Africa


In [12]:
alt.Chart(carbon_region[carbon_region['Year']>1950]).mark_rect().encode(
    x='Region:O',
    y='Year:O',
    color='Annual CO2 emissions:Q'
)

alt.Chart(...)

In [24]:
selection = alt.selection_multi(fields=['Region'], bind='legend')

alt.Chart(carbon_region).mark_area().encode(
    x="Year:Q",
    y=alt.Y("Annual CO2 emissions:Q", stack=True),
    color="Region:N",
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title='Annual CO2 emissions by region'
)

alt.Chart(...)

In [25]:
selection = alt.selection_multi(fields=['Region'], bind='legend')

alt.Chart(carbon_region).mark_area(opacity=0.2).encode(
    x="Year:Q",
    y=alt.Y("Annual CO2 emissions:Q", stack=None),
    color="Region:N",
    opacity=alt.condition(selection, alt.value(0.5), alt.value(0.2))
).add_selection(
    selection
).properties(
    title='Annual CO2 emissions by region'
)

alt.Chart(...)

In [33]:
selection = alt.selection_multi(fields=['Region'], bind='legend')

alt.Chart(carbon_region).mark_area().encode(
    x="Year:Q",
    y=alt.Y("Annual CO2 emissions:Q", stack="normalize"),
    color="Region:N",
    opacity=alt.condition(selection, alt.value(1), alt.value(0.3))
).add_selection(
    selection
).properties(
    title='Annual CO2 emissions by region normalized'
)

alt.Chart(...)

## 2.3) Emissões brasileiras

Os dados de emissão por país estão disponíveis na escala de milhões de toneladas.

In [39]:
carbon_brasil = carbon_country[carbon_country['Entity']=='Brazil']
carbon_brasil['Annual CO2 emissions (million t)'] = carbon_brasil['Annual CO2 emissions']

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Year'], empty='none')

# The basic line
line = alt.Chart(carbon_brasil).mark_line().encode(
    x='Year',
    y='Annual CO2 emissions (million t)'
).properties(
    title='Annual CO2 emissions from Brazil'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(carbon_brasil).mark_point().encode(
    x='Year',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='right', dx=-5, dy=-10).encode(
    text=alt.condition(nearest, 'Annual CO2 emissions (million t)', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(carbon_brasil).mark_rule(color='gray').encode(
    x='Year',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

# Parte 3 - Próximos passos

Em continuidade do trabalho, procurarei reproduzir alguns dos gráficos disponíveis no site, além de propor alguns outros.

Um dos gráficos que desejo reproduzir é este mapa que indica para cada país o quanto de carbono foi emitido em um certo ano, com o ano podendo ser escolhido pela pessoa que está visualizando. 

![title](annual-co2-emissions-per-country.png)

Outro gráfico que gostaria de reproduzir é este treemap que mostra a proporção de carbono emitida por cada continente/país no último ano com dados disponíveis.

![title](Annual-CO2-emissions-Treemap-1-603x550.png)